In [20]:
import requests
import pandas as pd
import json
from config import api_key
from bs4 import BeautifulSoup
from splinter import Browser
import time
import pymongo

In [2]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}

In [3]:
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# Most popular free to play games
baseURL = 'https://store.steampowered.com/search/?&genre=Free%20to%20Play'
browser.visit(baseURL)

In [23]:
# Scrape Functions
gameData = []
def scrape():
    section = browser.find_by_css('.search_result_row')

    urls = []
    for game in section:
        url = game['href']
        urls.append(url)

    for url in urls:

        # Visit game page
        browser.visit(url)
        time.sleep(1)
        
        if browser.find_by_css('.btnv6_blue_hoverfade').text == 'View Page':
            continue
        else:

            # Scrape game data
            try:
                name = browser.find_by_css('.apphub_AppName').value    
                release = browser.find_by_css('.date').value
                reviews = browser.find_by_css('.game_review_summary').value
            except:
                name = 'not found'
                release = 'not found'
                reviews = 'none'
            
            try:
                price = browser.find_by_css('.price').value
            except:
                price = browser.find_by_css('.discount_original_price').value



            # Get Dev / Genre / Clean
            try:
                dev = browser.find_by_css('.dev_row').value
                dev = dev.split('DEVELOPER:\n')
                dev = dev[1].split(',')[0]
            except:
                dev = 'Not Found'

            tags = browser.find_by_css('.app_tag')
            ignore = ['Free to Play','Multiplayer','Singleplayer']
            for t in tags:
                if t.value not in ignore:
                    genre = t.value
                    break

            # Get critic score if any
            try:
                criticScore = browser.find_by_css('.score').value
                criticScore
            except:
                criticScore = 'none'
            criticScore

            gameInfo = {
            'game':name,
            'price':price,
            'released':release,
            'reviews':reviews,
            'developer':dev,
            'criticScore':criticScore,
            'genre':genre}
            
            gameData.append(gameInfo)
            print(gameInfo)


            time.sleep(1)

            browser.visit(baseURL)
            
#     return gameData
    print(gameData)
        
    #     time.sleep(1)
    #     nextBtn = browser.find_by_text('>')
    #     nextBtn.click()
    #     baseURL = browser.url

        # print(f2p)

In [29]:
# Top Free to play games
baseURL = 'https://store.steampowered.com/search/?&genre=Free%20to%20Play'
browser.visit(baseURL)
scrape()

{'game': 'Counter-Strike: Global Offensive', 'price': 'Free to Play', 'released': 'Aug 21, 2012', 'reviews': 'Very Positive', 'developer': 'Valve', 'criticScore': '83', 'genre': 'FPS'}
{'game': 'Destiny 2', 'price': 'Free To Play', 'released': 'Oct 1, 2019', 'reviews': 'Very Positive', 'developer': 'Bungie', 'criticScore': '83', 'genre': 'Looter Shooter'}
{'game': 'Warframe', 'price': 'Free to Play', 'released': 'Mar 25, 2013', 'reviews': 'Very Positive', 'developer': 'Digital Extremes', 'criticScore': '69', 'genre': 'Looter Shooter'}
{'game': 'SMITE®', 'price': 'Free to Play', 'released': 'Sep 8, 2015', 'reviews': 'Very Positive', 'developer': 'Titan Forge Games', 'criticScore': '83', 'genre': 'MOBA'}
{'game': 'Path of Exile', 'price': 'Free to Play', 'released': 'Oct 23, 2013', 'reviews': 'Very Positive', 'developer': 'Grinding Gear Games', 'criticScore': '86', 'genre': 'Action RPG'}
{'game': 'War Thunder', 'price': 'Free to Play', 'released': 'Aug 15, 2013', 'reviews': 'Very Positiv

In [30]:
gameData

[{'game': 'Counter-Strike: Global Offensive',
  'price': 'Free to Play',
  'released': 'Aug 21, 2012',
  'reviews': 'Very Positive',
  'developer': 'Valve',
  'criticScore': '83',
  'genre': 'FPS',
  '_id': ObjectId('5e548fb7ff0590924f937f83')},
 {'game': 'Destiny 2',
  'price': 'Free To Play',
  'released': 'Oct 1, 2019',
  'reviews': 'Very Positive',
  'developer': 'Bungie',
  'criticScore': '83',
  'genre': 'Looter Shooter',
  '_id': ObjectId('5e548fb7ff0590924f937f84')},
 {'game': 'Warframe',
  'price': 'Free to Play',
  'released': 'Mar 25, 2013',
  'reviews': 'Very Positive',
  'developer': 'Digital Extremes',
  'criticScore': '69',
  'genre': 'Looter Shooter',
  '_id': ObjectId('5e548fb7ff0590924f937f85')},
 {'game': 'SMITE®',
  'price': 'Free to Play',
  'released': 'Sep 8, 2015',
  'reviews': 'Very Positive',
  'developer': 'Titan Forge Games',
  'criticScore': '83',
  'genre': 'MOBA',
  '_id': ObjectId('5e548fb7ff0590924f937f86')},
 {'game': 'Path of Exile',
  'price': 'Free 

In [31]:
# Connect to MongoDB 
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


db = client.steamDB
collection = db.scraped_f2p

# Insert data in database
collection.insert_many(gameData)

In [27]:
# Top Selling Games
baseURL = 'https://store.steampowered.com/search/?filter=globaltopsellers&os=win'
browser.visit(baseURL)
scrape()

{'game': 'RimWorld - Royalty', 'price': '$19.99', 'released': 'Feb 24, 2020', 'reviews': 'Very Positive', 'developer': 'Ludeon Studios', 'criticScore': 'none', 'genre': 'Indie'}
{'game': 'ARK: Genesis Season Pass', 'price': '$34.99', 'released': 'Aug 8, 2019', 'reviews': 'Mixed', 'developer': 'Studio Wildcard', 'criticScore': 'none', 'genre': 'Adventure'}
{'game': 'Wolcen: Lords of Mayhem', 'price': '$39.99', 'released': 'Feb 13, 2020', 'reviews': 'Mixed', 'developer': 'WOLCEN Studio', 'criticScore': '66', 'genre': 'RPG'}
{'game': 'not found', 'price': '$19.99', 'released': 'not found', 'reviews': 'none', 'developer': 'Paradox Development Studio', 'criticScore': 'none', 'genre': 'Strategy'}
{'game': 'Disco Elysium', 'price': '$34.98', 'released': 'Oct 15, 2019', 'reviews': 'Very Positive', 'developer': 'ZA/UM', 'criticScore': '91', 'genre': 'RPG'}
{'game': "PLAYERUNKNOWN'S BATTLEGROUNDS", 'price': '$29.99', 'released': 'Dec 21, 2017', 'reviews': 'Mixed', 'developer': 'PUBG Corporation'

In [28]:
# Connect to MongoDB 
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


db = client.steamDB
collection = db.scraped_data

# Insert data in database
collection.insert_many(gameData)